In [1]:
include("../LiPoSID.jl")

using LinearAlgebra
function ⊗(A::AbstractMatrix, B::AbstractMatrix)
    return kron(A,B)
end

function LME_operator_symbolic(H, J)
    
    Id = I(2)   
    U = -im*(Id ⊗ H - transpose(H) ⊗ Id)
    D = sum( 2*transpose(j')⊗j-Id⊗(j'*j) - transpose(j)*transpose(j')⊗Id for j in J )/2 
    
    Lᴸᴹᴱ = U + D
    
    return Lᴸᴹᴱ
end

function LME_operator_from_DMD(A)

    M = [ 0  0   1  1
          1  im  0  0 
          1  -im  0  0
          0  0  -1  1 ]/2
   
    Lᴰᴹᴰ =   M * A * inv(M)
   
    return Lᴰᴹᴰ 
   
end


LME_operator_from_DMD (generic function with 1 method)

In [65]:
using SymPy
@syms s

using DynamicPolynomials
using TSSOS

function Transfer_Function(A, b)
    inv(I(4)*s - A)*b
end

b⁰ = [0, 0, 1, 1]
b¹ = [0, 0, -1, 1]
bˣ = [1, 0, 0, 1]
bʸ = [0, 1, 0, 1]

function SumCoeffs2(poly)
    p = expand(numerator(poly))
    degrees = range(0,convert(Int32,SymPy.degree(p, s)))
    obj = sum(p.coeff(s^i)^2 for i in degrees)
    return obj
end

function sympy_to_dynamicpoly(sympy_expr::Sym)
    # Extract variables from the SymPy expression
    vars = free_symbols(sympy_expr)

    # Convert SymPy variables to string and then to Symbol
    var_symbols = [Symbol(string(v)) for v in vars]

    # Create DynamicPolynomials variables using @polyvar macro
    @eval begin
        @polyvar $(var_symbols...)
    end

    # Map SymPy variables to DynamicPolynomials variables
    var_map = Dict(vars[i] => eval(var_symbols[i]) for i in 1:length(vars))

    # Substitute using subs and convert to DynamicPolynomial
    dynamic_poly_expr = SymPy.subs(sympy_expr, var_map)
    dynamic_poly = eval(Meta.parse(string(dynamic_poly_expr)))
    return dynamic_poly
end

function TF_objective(Aˢʸᵐᵇ, Aˢⁱᵈ, b)

    Gˢʸᵐᵇ = Transfer_Function(Aˢʸᵐᵇ, b)[1:3]

    Gˢⁱᵈ = Transfer_Function(Aˢⁱᵈ, b)[1:3]

    ΔG = together.(Gˢʸᵐᵇ - Gˢⁱᵈ)

    polys = numerator.(ΔG)

    obj = sum(SumCoeffs2.(polys))

    return obj

end


TF_objective (generic function with 1 method)

In [58]:
ω = symbols("ω", real=true)
γ = symbols("γ", real=true)

Hᴸᴹᴱ = [ ω        0
         0        0   ]


Jᴸᴹᴱ = [ 0     γ
         0     0. + 0im  ]

Lᴸᴹᴱ = LME_operator_symbolic(Hᴸᴹᴱ, [Jᴸᴹᴱ])

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

Aᴸᴹᴱ = real.(inv(M) * Lᴸᴹᴱ * M)

Gᴸᴹᴱ = Transfer_Function(Aᴸᴹᴱ, bˣ)

4-element Vector{Sym{PyCall.PyObject}}:
 (1.0*s + 0.5*γ^2)/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
             1.0*ω/(1.0*s^2 + 1.0*s*γ^2 + 0.25*γ^4 + 1.0*ω^2)
                                1.0*γ^2/(1.0*s^2 + 1.0*s*γ^2)
                                                          1/s

In [59]:
data_dir = "../DATA/"
γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ_list_disp = [ "0.0795",  "0.2513", "0.7947", "2.5133", "7.9477", "25.133",  "79.477", "251.33"];

In [66]:
γᵢ = γ_list[8]
println(γᵢ)
γᶠ = parse(Float64, γᵢ)

ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

Lᴰᴹᴰ = LME_operator_from_DMD(Aᴰᴹᴰ⁻ˢᴮ)

M = [   0  0   1  1
        1  im  0  0 
        1 -im  0  0
        0  0  -1  1  ] / 2

@assert abs(LiPoSID.frobenius_norm2(inv(M) * Lᴰᴹᴰ * M - Aᴰᴹᴰ⁻ˢᴮ)) < 1e-9

obj = TF_objective(Aᴸᴹᴱ, Aᴰᴹᴰ⁻ˢᴮ,b¹) + TF_objective(Aᴸᴹᴱ, Aᴰᴹᴰ⁻ˢᴮ, bˣ) + TF_objective(Aᴸᴹᴱ, Aᴰᴹᴰ⁻ˢᴮ, bʸ)
#+ TF_objective(Aᴸᴹᴱ, Aᴰᴹᴰ⁻ˢᴮ, b⁰)

objective = sympy_to_dynamicpoly(obj)

best_solution, best_method = LiPoSID.sos_min_newton(objective) 

251.33


*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = -3.408126158810833e-10
Global optimality certified with relative optimality gap 0.000000%!
No higher TS step of the TSSOS hierarchy!
*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
optimum = -1.7735895296623752e-8
Global optimality certified with relative optimality gap 0.000002%!
No higher TS step of the TSSOS hierarchy!
val_p = [1.0023196629552456e-18, 1.002319633571578e-18]


(PolyVar{true}[γ, ω] => [1.6867171027249216e-13, 0.00012236234021161462], "scaled_tssos")

In [67]:
pop = [objective, γ-1, ω-1]
d = 4 # set the relaxation order
opt,sol,data = tssos_first(pop, variables(objective), d, numeq=1, TS="MD", solution = true)

*********************************** TSSOS ***********************************
Version 1.0.0, developed by Jie Wang, 2020--2023
TSSOS is launching...
Starting to compute the Gröbner basis...
This might take much time. You can set quotient=false to close it.
Starting to compute the block structure...
-----------------------------------------------------------------------------
The sizes of PSD blocks:
[3, 2]
[1, 2]
-----------------------------------------------------------------------------
Obtained the block structure in 0.000170975 seconds.
The maximal size of blocks is 3.
Assembling the SDP...
There are 9 affine constraints.
SDP assembling time: 0.000197665 seconds.
Solving the SDP...
Problem
  Name                   :                 
  Objective sense        : maximize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 9               
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones       

(-7.583508764037743e-13, [1.1723825091406097, 1.0], TSSOS.cpop_data(2, 0, 1, 1, PolyVar{true}[γ, ω], Polynomial{true, Float64}[6.000389721153201e-20γ⁸ + 1.8834009018517946e-20γ⁶ + 4.800311776922561e-19γ⁴ω² - 1.1776940768241963e-22γ⁴ω + 6.935061010989177e-19γ⁴ + 7.533603607407176e-20γ²ω² + 9.600623553845122e-19ω⁴ + 5.187960087170326e-22γ²ω - 4.710776307296777e-22ω³ + 4.808695713453694e-19γ² + 1.8963963852410965e-18ω² - 4.721840225230717e-22ω + 1.0023196629552456e-18, γ - 1.0, ω - 1.0], Polynomial{true, Float64}[ω - 1.0], UInt8[0x00; 0x01;;], Matrix{UInt8}[Matrix{UInt8}(undef, 2, 0), [0x01 0x00; 0x00 0x00]], [Float64[], [1.0, -1.0]], Matrix{UInt8}[[0x00 0x01 … 0x03 0x04; 0x00 0x00 … 0x00 0x00], [0x00 0x01 0x02 0x03; 0x00 0x00 0x00 0x00]], UInt8[0x00 0x02 … 0x01 0x02; 0x00 0x00 … 0x00 0x00], [3, 2], [1, 2], Vector{Vector{UInt16}}[[[0x0001, 0x0003, 0x0005], [0x0002, 0x0004], [0x0001, 0x0002]], [[0x0001, 0x0002, 0x0003, 0x0004]]], UInt16[0x0003, 0x0001], Vector{UInt16}[[0x0003, 0x0002, 0x00

In [68]:
sol

2-element Vector{Float64}:
 1.1723825091406097
 1.0

In [ ]:
opt,sol,data = tssos_higher!(data, TS="MD")

In [6]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1  1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

max_negs = []
sum_negs = []
num_negs = []

for γᵢ in γ_list 

    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γᵢ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γᵢ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γᵢ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γᵢ)

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])

    objective = TF_objective(Aᴸᴹᴱ, Aᴰᴹᴰ⁻ˢᴮ, bₓ)

    best_solution, best_method = LiPoSID.sos_min_newton(objective) 

end

UndefVarError: UndefVarError: `bₓ` not defined